In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Loading Data

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
from tqdm.notebook import tqdm_notebook

In [ ]:
train5 = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean5L.csv')
train10 = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean10L.csv')
train15 = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean15L.csv')
train20 = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean20L.csv')
train25 = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean25L.csv')
trainlast = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean_Last.csv')


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/test_clean.csv')

In [ ]:
test.isnull().sum()

PRODUCT_ID    0
TEXT          0
dtype: int64

In [ ]:
data = [train5,train10,train15,train20,train25,trainlast]
names = ['train5','train10','train15','train20','train25','trainlast']

# Checking Data and Handling Null Values

In [ ]:
for df,name in zip(data, names):
  print(name)
  display(df.info())
  print()

train5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494187 entries, 0 to 494186
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   BROWSE_NODE_ID  494187 non-null  int64 
 1   TEXT            494171 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.5+ MB


None


train10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493974 entries, 0 to 493973
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   BROWSE_NODE_ID  493974 non-null  int64 
 1   TEXT            493964 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.5+ MB


None


train15
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494042 entries, 0 to 494041
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   BROWSE_NODE_ID  494042 non-null  int64 
 1   TEXT            494025 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.5+ MB


...

None


train25
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493957 entries, 0 to 493956
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   BROWSE_NODE_ID  493957 non-null  int64 
 1   TEXT            493944 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.5+ MB


None


trainlast
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398771 entries, 0 to 398770
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   BROWSE_NODE_ID  398771 non-null  int64 
 1   TEXT            398764 non-null  object
dtypes: int64(1), object(1)
memory usage: 6.1+ MB


None

In [ ]:
for df,name in zip(data, names):
  print(name)
  display(df.isnull().sum())
  print()

train5


BROWSE_NODE_ID     0
TEXT              16
dtype: int64


train10


BROWSE_NODE_ID     0
TEXT              10
dtype: int64


train15


...

BROWSE_NODE_ID     0
TEXT              13
dtype: int64


train25


BROWSE_NODE_ID     0
TEXT              13
dtype: int64


trainlast


BROWSE_NODE_ID    0
TEXT              7
dtype: int64

In [ ]:
for df,name in zip(data, names):
  print(name)
  df.dropna(inplace=True)

train5
train10
train15
train20
train25
trainlast


#Corpus

In [ ]:
def build_vocab(sentences):
  word_freq = {}
  for row in tqdm(sentences):
      for word in row.split():
        if len(word)>2:
          try:
            word_freq[word] += 1
          except:
            word_freq[word] = 1
  
  print("Total words found:",len(word_freq))
  return word_freq

In [ ]:
train_vocab = {}
for df,name in zip(data, names):
  print(name)
  vocab = build_vocab(df['TEXT'])
  train_vocab.update(vocab)
  print()

print(len(train_vocab))

  1%|          | 3861/494171 [00:00<00:12, 38602.95it/s]

train5


  1%|          | 2781/493964 [00:00<00:17, 27809.83it/s]

Total words found: 259805

train10


  1%|          | 2742/494025 [00:00<00:17, 27415.65it/s]

...

Total words found: 259314

train25


  1%|          | 3104/398764 [00:00<00:12, 31019.10it/s]

Total words found: 259352

trainlast


100%|██████████| 398764/398764 [00:15<00:00, 25054.46it/s]


Total words found: 230956

645742


In [ ]:
test_vocab = build_vocab(test['TEXT'])

100%|██████████| 110775/110775 [00:05<00:00, 21315.20it/s]

Total words found: 79095


#NLTK and Gensim

https://radimrehurek.com/gensim/models/word2vec.html

https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

In [ ]:
import gensim
import gensim.downloader
from gensim.models import KeyedVectors

In [ ]:
#list(gensim.downloader.info()['models'].keys())

In [ ]:
#word2vec_model = gensim.downloader.load('word2vec-google-news-300') 
#word2vec_model.save_word2vec_format('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/model.bin', binary=True)

[=================================================-] 99.8% 1660.0/1662.8MB downloaded


In [ ]:
#model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/wordvec.bin', binary=True)
#len(model.vocab)

3000000

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2021-08-01 09:18:20--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-08-01 09:18:20--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.09MB/s    in 2m 40s  

2021-08-01 09:21:00 (5.13 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!unzip /content/glove.6B.zip

Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
# load the whole embedding into memory
glove_embeddings = dict()
f = open('/content/glove.6B.300d.txt')

for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embeddings[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(glove_embeddings))

400000it [00:32, 12449.54it/s]

Loaded 400000 word vectors.


#checking vocab in word2vec

In [ ]:
def check_coverage(embeddings, vocab):
  words_to_remove = {}
  for word in tqdm(list(vocab.keys())):
    try:
      embeddings[word]
    except:
      words_to_remove[word] = 0

  print(len(words_to_remove))
  return words_to_remove

In [ ]:
train_words_to_remove = check_coverage(glove_embeddings, train_vocab)

100%|██████████| 645742/645742 [00:00<00:00, 813156.31it/s]


529959


In [ ]:
test_words_to_remove = check_coverage(glove_embeddings, test_vocab)

100%|██████████| 79095/79095 [00:00<00:00, 856332.83it/s]

40491


# Combining and exporting data

In [ ]:
train = pd.concat([train5,train10,train15,train20,train25,trainlast], ignore_index=True)
print(train.shape)

(2868897, 2)


In [ ]:
train.head(3)

,BROWSE_NODE_ID,TEXT
0,0,pete cat bedtime blue doll inch pete cat coole...
1,1,new yorker nyhm refrigerator magnet new yorker...
2,2,ultimate self sufficiency handbook complete gu...


In [ ]:
train.to_csv('/content/drive/MyDrive/Datasets/Amazon_ML_Challenge/Dataset/train_clean.csv', index=False)

# END